# Introduction to RL Agent

This notebook introduces the basics of reinforcement learning agents.

## Setup

Install required dependencies for the RL environment.

## Import Libraries

In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## Create Environment

In [3]:
# Create the LunarLander-v3 environment
env = gym.make("LunarLander-v3")
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

Observation space: Box([ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ], [ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ], (8,), float32)
Action space: Discrete(4)


## Define the PPO Model

In [11]:
# Create the PPO model
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    device='cuda'
)

print("Model created successfully!")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Model created successfully!


/home/harris/miniconda3/envs/huggingface_rl/lib/python3.13/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


## Train the Model

In [12]:
# Train the agent for 1 million timesteps
total_timesteps = 1000
model.learn(total_timesteps=total_timesteps)

# Save the trained model
model.save("ppo_lunarlander")
print("Training complete! Model saved.")

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 92.9     |
|    ep_rew_mean     | -180     |
| time/              |          |
|    fps             | 878      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
Training complete! Model saved.


## Evaluate the Trained Model

In [13]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

/home/harris/miniconda3/envs/huggingface_rl/lib/python3.13/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: -275.61 +/- 200.31


## Visualize the Results

In [17]:
# Record a video of the trained agent
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='ppo-lunarlander'):
    """
    Record a video of the agent
    """
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    
    # Start recording
    eval_env = VecVideoRecorder(eval_env, f"videos/{prefix}",
                               record_video_trigger=lambda x: x == 0, video_length=video_length,
                               name_prefix=prefix)
    
    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs, deterministic=True)
        obs, _, _, _ = eval_env.step(action)
    
    # Close the video recorder
    eval_env.close()

record_video('LunarLander-v3', model, video_length=1000)
print("Video recorded successfully!")

Saving video to /home/harris/dev/huggingface_rl/tutorials/videos/ppo-lunarlander/ppo-lunarlander-step-0-to-step-1000.mp4
MoviePy - Building video /home/harris/dev/huggingface_rl/tutorials/videos/ppo-lunarlander/ppo-lunarlander-step-0-to-step-1000.mp4.
MoviePy - Writing video /home/harris/dev/huggingface_rl/tutorials/videos/ppo-lunarlander/ppo-lunarlander-step-0-to-step-1000.mp4



MoviePy - Done !
MoviePy - video ready /home/harris/dev/huggingface_rl/tutorials/videos/ppo-lunarlander/ppo-lunarlander-step-0-to-step-1000.mp4
Video recorded successfully!
